In [1]:
import torch 
import torchvision 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader

In [2]:
epochs=10

device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=1),
    transforms.RandomAffine(30),
    transforms.RandomCrop((20, 20)),
    transforms.RandomResizedCrop((22, 22)),
    transforms.RandomPerspective(),
    transforms.RandomRotation(90, expand=False),
    transforms.ColorJitter(brightness=(0.2, 2),
                           contrast=(0.3, 2), 
                           saturation=(0.2, 2),
                           hue=(-0.3, 0.3)),
    transforms.Resize(227),
    transforms.ToTensor()
 ])

test_transform = transforms.Compose([
    transforms.Resize(227),
    transforms.ToTensor()
    ])
train_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(train_data,batch_size=64,shuffle=True)

test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

testloader = DataLoader(test_data,batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class VGG19(nn.Module):
  def __init__(self):
    super(VGG19,self).__init__()

    self.conv_layers = nn.Sequential(
        nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        #nn.ReLU(inplace=True),
        nn.SiLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Linear(512*7*7,4096),
        nn.Dropout(0.2),
        nn.ReLU(inplace=True),
        nn.Linear(4096,4096),
        nn.Dropout(0.2),
        nn.ReLU(inplace=True),
        nn.Linear(4096,100)
    )

  def forward(self,x):
    output = self.conv_layers(x)
    output = output.view(output.size(0), -1)
    output = self.classifier(output)

    return output

In [5]:
model = VGG19().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9,weight_decay=0.0005)
#optimizer = optim.AdamW(model.parameters(),lr=0.0001)
print(model)

VGG19(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): SiLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): SiLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): SiLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): SiLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): SiLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): SiLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): SiLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1),

In [6]:
train_accu = []
train_losses = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    running_loss=0
    correct=0
    total=0

    for batch_idx, (data, target) in enumerate(train_loader):
        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() 
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step() 

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_loss=running_loss/len(train_loader)
    accu=100.*correct/total
   
    train_accu.append(accu)
    train_losses.append(train_loss)
    
    print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))

In [7]:
eval_losses=[]
eval_accu=[]

def test(model, device, test_loader):
    model.eval() 
    running_loss=0
    correct=0
    total=0
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            running_loss += loss_fn(output, target).sum().item()
            pred = output.max(1, keepdim=True)[1]
            total += target.size(0)
            correct += pred.eq(target.view_as(pred)).sum().item() 

        test_loss = running_loss/len(test_loader.dataset)  
        accu=100.*correct/total
 
        eval_losses.append(test_loss)
        eval_accu.append(accu)
        print('Test Loss: %.3f | Accuracy: %.3f'%(test_loss,accu)) 
        print('-----------------------------------')

In [8]:
for epoch in range(1, epochs+1):
  print(f"Epoch {epoch}")
  train(model, device, trainloader, optimizer, epoch)
  test(model, device, testloader)

Epoch 1
Train Loss: 4.605 | Accuracy: 0.944
Test Loss: 0.072 | Accuracy: 1.000
-----------------------------------
Epoch 2
Train Loss: 4.605 | Accuracy: 0.940
Test Loss: 0.072 | Accuracy: 1.000
-----------------------------------
Epoch 3
Train Loss: 4.605 | Accuracy: 0.946
Test Loss: 0.072 | Accuracy: 1.000
-----------------------------------
Epoch 4


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt


plt.plot(train_accu)
plt.plot(eval_accu)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Test'])
plt.title('Train vs Test Accuracy')
 
plt.show()

In [ ]:
plt.plot(train_losses)
plt.plot(eval_losses)
plt.xlabel('epoch')
plt.ylabel('losses')
plt.legend(['Train','Test'])
plt.title('Train vs Test Losses')
 
plt.show()

In [ ]:
classes = ['airplane', 'automobile', 'bird','cat','deer', 'dog', 'frog','horse','ship','truck']

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device),target.to(device)
        outputs = model(data)
        _, predictions = torch.max(outputs, 1)

        for label, prediction in zip(target, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')